## TransactionsAll script
### Query and reports for Project 1 
#### Submitted by Student Thanh Nam Tran for CMSC461@UMBC (Fall '2020)



In [1]:
!pip install mysql-connector-python

In [2]:
import mysql.connector

In [3]:
# get DB username and password from the standard input
import getpass
db_user = getpass.getpass('Enter username')
db_password = getpass.getpass('Enter password')

Enter username········
Enter password········


In [4]:
# prepare the configuration parameters for connecting to database
config = {
   'user': db_user,
   'password': db_password,
   'host': 'localhost',
   'port': 3306,
   'database': 'pluto_realty',
   'raise_on_warnings': True                    
}


In [5]:
#connect to DB server
db_connection = mysql.connector.connect(**config)

In [6]:
#open a cursor (iterator) for stepping through the tuples in the resultset of an SQL query 
cur = db_connection.cursor()

In [7]:
#Below are the required queries:
#The queries are implementing and printing using pandas 

import pandas as pd

In [8]:
#1. List the names of all the unique clients.
df = pd.read_sql('SELECT client_id FROM clients', con=db_connection)
print(df)


   client_id
0      00007
1      00008
2      00009
3      00010
4      00011
5      00012
6      00013
7      00014
8      00015
9      00016
10     00017
11     00018
12     00019
13     00020
14     00021


In [9]:
#2. Find the unique names of owners and total square footage of all the properties they own
df2 = pd.read_sql('SELECT distinct first_name, sum(rental_properties.area_sqr_footage) from property_owner inner join rental_properties on property_owner.owner_id = rental_properties.owner_id  group by first_name order by first_name ', con=db_connection)
print(df2)

   first_name  sum(rental_properties.area_sqr_footage)
0      Anabia                                  11576.0
1        Anna                                   8000.0
2      Bianka                                   2224.0
3       Faiza                                    223.0
4       Henna                                   3000.0
5      Henna                                     323.0
6        Izaa                                   6268.0
7      Maisie                                   8443.0
8      Marlie                                   8223.0
9     Philip                                   17525.0
10     Samira                                   6523.0


In [10]:
#3. Find the properties shown by each associate in a given month. 
#Let the given month be March change parameter in the command below to see other month
df3 = pd.read_sql('SELECT associates.id as Asscociate_ID, rental_properties.id as Rental_property_ID, prop_date_time as TIME FROM associates inner join rental_properties where associates.property_id = rental_properties.id and MONTH(prop_date_time) = "03"', con=db_connection)
print(df3)


  Asscociate_ID Rental_property_ID       TIME
0         00001              10002 2014-03-26
1         00002              20001 2014-03-26
2         00003              20003 2014-03-26
3         00001              30001 2014-03-26


In [11]:
#4. Find the most popular properties (in terms of number of viewings in a given year).
#Use year 2019 change parameter in the command below to see other years
df4 = pd.read_sql('SELECT property_viewings.property_id, count(prop_view_id) as count FROM pluto_realty.prop_view_date inner join property_viewings on prop_view_id = property_viewing_id inner join rental_properties on rental_properties.id = property_viewings.property_id where YEAR(prop_view_date.date_time) = "2019" group by prop_view_id order by count desc LIMIT 1;', con=db_connection)
print(df4)

  property_id  count
0       30006      2


In [12]:
#5. Find the total rent month due to each property owner.
df5= pd.read_sql('SELECT property_owner.property_id, rent_month FROM pluto_realty.property_owner inner JOIN rental_properties on property_owner.property_id = rental_properties.id;', con=db_connection)
print(df5)

   property_id  rent_month
0        10001      3000.0
1        10002      4000.0
2        10003      2000.0
3        10004      2200.0
4        20001      5000.0
5        20002      4000.0
6        20003      6000.0
7        20004      7200.0
8        30001      1000.0
9        30002      1000.0
10       30003      1000.0
11       30004      1200.0
12       10005      3300.0
13       10006      2500.0
14       20005      6000.0
15       20006      7200.0
16       30005      1200.0
17       30006      1200.0


In [13]:
#6. Find the unique names of associates supervised (directly or indirectly) by a given employee
#Example: Let the supervior id be 00002, find unique names of associates supervised by 00002
# change parameter in the command below to see other supervisor
df6 = pd.read_sql('SELECT emp_id FROM pluto_realty.supervisor where spv_id =  "00002" ', con=db_connection)
print(df6)


  emp_id
0  00001
1  00004
2  00005
3  00006


In [14]:
#7. Find the unique names of owners that have a residential property in every city where Pat Doe
# owns a commercial property.
df7 = pd.read_sql('SELECT rental_properties.owner_id FROM pluto_realty.rental_properties inner join residential_prop on rental_properties.id = residential_prop.id inner join commercial_prop on rental_properties.id = commercial_prop.id inner join property_owner on property_owner.owner_id = rental_properties.owner_id where residential_prop.city = commercial_prop.city and first_name = "Pat" and last_name = "Doe" ', con=db_connection)
print(df7)



Empty DataFrame
Columns: [owner_id]
Index: []


In [15]:
#8. Find the top-3 partners with respect to number of properties leased in the current year.
#Use year 2020  change parameter in the command below to see other years
df8 = pd.read_sql('SELECT id, count(id) FROM pluto_realty.partners  inner join leases  on partners.lease_id = leases.lease_id where leases.start >= "2019-00-00" and leases.finish <= "2020-00-00" group by id', con=db_connection)
print(df8)


      id  count(id)
0  00004          2
1  00005          2
2  00006          1


In [17]:
#9. Write a SQL function to compute the total management fees due to Pluto in the last 3 months

# NOTE: IMPORTANT need to do this command before create function:
trust = ('SET GLOBAL log_bin_trust_function_creators = 1;')
cur.execute(trust)
func = ('CREATE FUNCTION `3month_management_fee` (rent_fee float) RETURNS INTEGER BEGIN 	DECLARE cal_total float; 	DECLARE total float;   SELECT sum(rent_fee) as cal_total INTO cal_total from rental_properties;    SET total = 3*cal_total; RETURN total; END')
cur.execute(func)        

In [ ]:
#10. Create a SQL trigger to automatically set to FALSE the advertisement flag of a property when it is
# leased.
DELIMITER USE `pluto_realty` DROP TRIGGER IF EXISTS advertisement; CREATE TRIGGER advertisement AFTER INSERT ON `rental_properties` FOR EACH ROW BEGIN 	If lease_id IS NULL then SET status = 0; END$$ DELIMITER ;

In [16]:
#close the cursor
cur.close()

True

In [17]:
#close the connection to the DB server
db_connection.close()

In [91]:
#capture host information
import datetime
print(datetime.datetime.now())
!whoami
!hostname
#!ipconfig | grep 'IPv4 Address'
#!ipconfig | grep 'DNS'
!ipconfig
%connect_info

2020-12-12 22:54:04.228016
desktop-38srboc\your daddy
DESKTOP-38SRBOC

Windows IP Configuration


Ethernet adapter Ethernet:

   Media State . . . . . . . . . . . : Media disconnected
   Connection-specific DNS Suffix  . : 

Wireless LAN adapter Local Area Connection* 2:

   Media State . . . . . . . . . . . : Media disconnected
   Connection-specific DNS Suffix  . : 

Wireless LAN adapter Local Area Connection* 3:

   Media State . . . . . . . . . . . : Media disconnected
   Connection-specific DNS Suffix  . : 

Wireless LAN adapter Wi-Fi:

   Connection-specific DNS Suffix  . : 
   Link-local IPv6 Address . . . . . : fe80::c852:7814:4dbd:8bc5%20
   IPv4 Address. . . . . . . . . . . : 192.168.200.32
   Subnet Mask . . . . . . . . . . . : 255.255.255.0
   Default Gateway . . . . . . . . . : 192.168.200.1

Ethernet adapter Bluetooth Network Connection 3:

   Media State . . . . . . . . . . . : Media disconnected
   Connection-specific DNS Suffix  . : 
{
  "shell_port": 50122,
  "iopub_p